In [1]:
import os
import time
import httpx 

import thetadata
from thetadata import ThetaClient
from dotenv import load_dotenv

In [2]:
# Get environment variables in .env in project root
load_dotenv()

USERNAME=os.getenv('THETAUSER')
PASSWORD=os.getenv('THETAPASS')

In [3]:
client = ThetaClient(username=USERNAME, passwd=PASSWORD)
time.sleep(8) # TODO because it isn't started yet. Need to fix

If you require API support, feel free to join our discord server! http://discord.thetadata.us
[10-27-2024 10:18:22] INFO: Starting Theta Terminal v1.6.7 Revision A...
[10-27-2024 10:18:22] INFO: Config dir: C:\Users\david\ThetaData\ThetaTerminal
[10-27-2024 10:18:23] INFO: [MDDS] Attempting login as davidmckim@gmail.com
[10-27-2024 10:18:23] INFO: [FPSS] Attempting login as davidmckim@gmail.com
[10-27-2024 10:18:26] INFO: [MDDS] CONNECTED: [nj-a.thetadata.us:12000], Bundle: STOCK.FREE, OPTION.PRO, INDEX.FREE
[10-27-2024 10:18:26] INFO: [FPSS] CONNECTED: [nj-a.thetadata.us:20000], Bundle: STOCK.FREE, OPTION.PRO, INDEX.FREE


In [3]:
from pydantic import BaseModel, Field, field_validator
from datetime import date
from typing import List
import pandas as pd

class ExpirationsResponse(BaseModel):
    """Response model for option expirations data.

    The response contains a list of expiration dates in YYYYMMDD format
    """
    expirations: List[int] = Field(
        ...,
        description="List of expiration dates in YYYYMMDD format"
    )

    @field_validator('expirations')
    @classmethod
    def validate_expirations(cls, v: List[int]) -> List[int]:
        """Validate each expiration date."""
        for exp in v:
            str_date = str(exp)
            if len(str_date) != 8:
                raise ValueError(f"Expiration {exp} not in YYYYMMDD format")
            try:
                year = int(str_date[:4])
                month = int(str_date[4:6])
                day = int(str_date[6:8])
                date(year, month, day)
            except ValueError:
                raise ValueError(f"Invalid expiration date: {exp}")
        return sorted(v)  # Return sorted list for consistency

    def to_pandas(self) -> pd.Series:
        """Convert response to pandas Series with datetime index."""
        return pd.Series(
            index=pd.to_datetime(self.expirations, format='%Y%m%d'),
            data=self.expirations,
            name="expirations"
        )

In [31]:
try:
    port=25510
    host="127.0.0.1"
    
    url = f"http://{host}:{port}/v2/list/expirations"
    
    params = {
        'root': 'AAPL'
    }

    # Make request with error handling
    response = httpx.get(
        url=url,
        params=params,
    ).raise_for_status()
    
    # get the headers and validate with pydantic model
    headers = response.headers
    next_page = headers['next-page']
    latency = headers['latency']
    content_type = headers['content-type']
    content_length = headers['content-length']
    
    if next_page == 'null':
        # we are finished and can return data
        pass
    else:
        # we need to keep making requests before we can return data
        pass
    
    
    # Get the response data, validate with pydantic, this will be returned to the user
    data = response.json()['response']
except Exception as e:
    print(e)

In [30]:
headers

Headers({'access-control-allow-headers': '*', 'access-control-request-headers': '*', 'date': 'Sun, 08 Dec 2024 17:50:02 GMT', 'latency': '62', 'content-type': 'application/json', 'access-control-allow-origin': 'https://http-docs.thetadata.usFPSS_QUEUE_DEPTH = 100000', 'next-page': 'null', 'content-length': '8156'})

In [21]:
data

[20120601,
 20120608,
 20120616,
 20120622,
 20120629,
 20120706,
 20120713,
 20120721,
 20120727,
 20120803,
 20120810,
 20120818,
 20120824,
 20120831,
 20120907,
 20120914,
 20120922,
 20120928,
 20121005,
 20121012,
 20121020,
 20121026,
 20121102,
 20121109,
 20121117,
 20121123,
 20121130,
 20121207,
 20121214,
 20121222,
 20121228,
 20130104,
 20130111,
 20130119,
 20130125,
 20130201,
 20130208,
 20130216,
 20130222,
 20130301,
 20130308,
 20130316,
 20130322,
 20130328,
 20130405,
 20130412,
 20130420,
 20130426,
 20130503,
 20130510,
 20130518,
 20130524,
 20130531,
 20130607,
 20130614,
 20130622,
 20130628,
 20130705,
 20130712,
 20130720,
 20130726,
 20130802,
 20130809,
 20130817,
 20130823,
 20130830,
 20130906,
 20130913,
 20130921,
 20130927,
 20131004,
 20131011,
 20131019,
 20131025,
 20131101,
 20131108,
 20131116,
 20131122,
 20131129,
 20131206,
 20131213,
 20131221,
 20131227,
 20140103,
 20140110,
 20140118,
 20140124,
 20140131,
 20140207,
 20140214,
 20140222,

In [34]:
import httpx  # install via pip install httpx
import csv

BASE_URL = "http://127.0.0.1:25510/v2"  # all endpoints use this URL base

# set params
params = {
	'root': 'AAPL',
	'use_csv': 'true',
	'ivl': '60000',
	'start_date': '20241107',
	'end_date': '20241107',
}


#
# This is the streaming version, and will read line-by-line
#
url = BASE_URL + '/hist/stock/quote'

while url is not None:
    with httpx.stream("GET", url, params=params) as response:
        response.raise_for_status()  # make sure the request worked
        headers = response.headers
        print(headers)
        for line in response.iter_lines():
            print(line)  # do something with the data

    # check the Next-Page header to see if we have more data
    if 'Next-Page' in response.headers and response.headers['Next-Page'] != "null":
        url = response.headers['Next-Page']
    else:
        url = None

Headers({'access-control-allow-headers': '*', 'content-disposition': 'attachment; filename=QUOTE.E-AAPL.20241107-20241107.60000.csv;', 'access-control-request-headers': '*', 'date': 'Sun, 08 Dec 2024 17:59:44 GMT', 'latency': '95', 'content-type': 'text/csv', 'access-control-allow-origin': 'https://http-docs.thetadata.usFPSS_QUEUE_DEPTH = 100000', 'next-page': 'null', 'content-length': '19521'})
ms_of_day,bid_size,bid_exchange,bid,bid_condition,ask_size,ask_exchange,ask,ask_condition,date
34200000,1,1,224.5300,0,8,1,224.6500,0,20241107
34260000,5,1,225.6400,0,2,1,225.6500,0,20241107
34320000,2,1,225.9200,0,3,7,225.9400,0,20241107
34380000,7,1,226.5200,0,1,1,226.5400,0,20241107
34440000,4,1,226.3200,0,7,75,226.3600,0,20241107
34500000,1,1,225.8700,0,7,75,225.9000,0,20241107
34560000,2,7,225.9400,0,7,60,225.9700,0,20241107
34620000,1,73,225.9100,0,1,1,225.9200,0,20241107
34680000,2,1,225.7600,0,3,7,225.7800,0,20241107
34740000,1,1,225.9900,0,2,1,226.0000,0,20241107
34800000,1,65,225.8500